<a href="https://colab.research.google.com/github/asifahsaan/Deep-Learning/blob/main/Full_Image_Classification_Workflow_Recreated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Complete Beginner's Notebook: Image Preprocessing, Classification & Prediction
This notebook walks you through the entire image classification pipeline using TensorFlow and Keras.
From loading data, preprocessing, training a CNN model, to evaluating and predicting new images.

## 📦 Step 1: Import Required Libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
from PIL import Image

## 🗂️ Step 2: Load the Flower Dataset

In [ ]:
import pathlib
dataset_url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
data_dir = tf.keras.utils.get_file(origin=dataset_url, fname='flower_photos', untar=True)
data_dir = pathlib.Path(data_dir)
print("Dataset loaded from:", data_dir)

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Dataset loaded from: /root/.keras/datasets/flower_photos


## 🖼️ Step 3: View a Sample Image

In [ ]:
# # Download the dataset again
# dataset_url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
# data_dir = tf.keras.utils.get_file(origin=dataset_url,
#                                    fname='flower_photos',
#                                    untar=True)
# data_dir = pathlib.Path(data_dir)

# Check the contents
for folder in data_dir.iterdir():
    print(f"{folder.name}: {len(list(folder.glob('*')))} images")

flower_photos: 6 images


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Try to load a sample image from the first available class
for class_folder in data_dir.iterdir():
    # Filter for common image file extensions
    image_files = [f for f in class_folder.glob("*") if f.suffix in ['.jpg', '.jpeg', '.png', '.bmp']]
    if image_files:
        sample_img_path = image_files[0]
        try:
            img = Image.open(sample_img_path)
            plt.imshow(img)
            plt.title(f"Sample Image - {class_folder.name}")
            plt.axis('off')
            plt.show()
            break # Stop after finding and displaying one image
        except UnidentifiedImageError:
            print(f"Could not identify image file: {sample_img_path}")
            continue # Skip to the next file if identification fails

else:
    print("No image files found in any category.")

No image files found in any category.


## 🧹 Step 4: Resize and Normalize the Image

In [ ]:
img_resized = img.resize((128, 128))
img_array = np.array(img_resized) / 255.0
print("Resized shape:", img_array.shape)
plt.imshow(img_array)
plt.title("Resized and Normalized Image")
plt.axis('off')
plt.show()

## 📂 Step 5: Load Dataset for Training & Validation

In [ ]:
batch_size = 32
img_height = 128
img_width = 128
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
class_names = train_ds.class_names
print("Class names:", class_names)

## ⚙️ Step 6: Optimize Dataset Performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## 🧠 Step 7: Define the CNN Model

In [ ]:
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names))
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

## 🚀 Step 8: Train the Model

In [ ]:
epochs = 5
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

## 📊 Step 9: Evaluation Report

In [ ]:
y_true = np.concatenate([y for x, y in val_ds], axis=0)
y_pred_probs = model.predict(val_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
print(classification_report(y_true, y_pred, target_names=class_names))

## 🔮 Step 10: Predict a New Image

In [ ]:
def predict_image(image_path):
    img = tf.keras.utils.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    predicted_class = class_names[np.argmax(score)]
    confidence = 100 * np.max(score)
    print(f"Predicted: {predicted_class} with {confidence:.2f}% confidence")

# Example:
# predict_image('/path/to/image.jpg')